## Graph Neural Networks with Pytorch
## Target: Simplifying Graph Convolutional Networks
- Original Paper: https://arxiv.org/abs/1902.07153
- Original Code: https://github.com/rusty1s/pytorch_geometric/blob/master/examples/sgc.py

In [1]:
import os
import sys

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import SGConv

sys.path.append('../')
from utils import *
logger = make_logger(name='sg_logger')

In [2]:
# Load Dataset
dataset = 'Cora'
path = os.path.join(os.getcwd(), '..', 'data', dataset)
dataset = Planetoid(path, dataset)
data = dataset[0]

In [3]:
class SG(torch.nn.Module):
    def __init__(self):
        super(SG, self).__init__()
        self.conv1 = SGConv(
            dataset.num_features, dataset.num_classes, K=2, cached=True)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        return F.log_softmax(x, dim=1)

**Simplfied Graph Convolutional Networks**는 기존의 **GCN**에서 많은 시간을 소요하게 만들었던 활성화 함수를 상당 부분 생략함으로써,  
속도 향상과 더불어 성능 유지라는 두마리 토끼를 다 잡을 수 있다는 사실을 증명한 방법론이다.  

**SGConv** Layer는 아래와 같이 구성된다.  

$$ \mathbf{H}^{`} = (\mathbf{D}^{-0.5} \hat{\mathbf{A}} \mathbf{D}^{0.5})^{K} \mathbf{H} \mathbf{W} $$  

위 식에서 $K$ 는 총 Layer의 수를 의미한다.  

`Adjacency Matrix`를 활용한 모든 계산을 미리 끝낼 수 있기 때문에,  
앞 부분을 미리 계산 한 뒤 이를 캐싱해두면 계산 시간을 크게 단축시킬 수 있다.  

이러한 계산 생략에도 불구하고 본 알고리즘은 상당한 성능을 보여주었으며 이에 대한 자세한 내용은 [논문 원본](https://arxiv.org/abs/1902.07153)을 참조하길 바란다.  

`in_channels`와 `out_channels` 인자는 Input/Output의 Size를 의미한다.  

`K` 인자는 앞서 설명하였듯이 총 Layer의 수 혹은 hops의 수를 의미한다. 기본 값은 1이다.  

`cached` 인자의 기본 값은 `False`인데, `True`로 설정하면, 첫 수행 후 행렬 계산 식의 결과 값을 캐싱하여 추후에 사용하게 된다.  

이 경우 시간을 단축할 수 있지만 오직 **Transductive** 학습 상황에서만 가능하다.  

왜냐하면 만약 **Inductive**한 상황, 즉 새로운 Node가 진입할 수 있다면 행렬 계산 식의 결과 값을 캐싱하는 것은 의미가 없기 때문이다.  

`add_self_loops` 인자와 `bias` 인자는 다른 Graph Conv Layer들과 마찬가지로 Input Graph에 Self-loop를 추가할지 여부와 bias 추가 여부를 의미한다.  

[공식 문서](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.SGConv)를 참조하면 Source 코드 또한 참고할 수 있다.  

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = SG().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=0.005)


def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [5]:
best_val_acc = test_acc = 0
for epoch in range(1, 51):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, train_acc, best_val_acc, test_acc))

Epoch: 001, Train: 0.9571, Val: 0.7020, Test: 0.7060
Epoch: 002, Train: 0.9714, Val: 0.7560, Test: 0.7950
Epoch: 003, Train: 1.0000, Val: 0.7740, Test: 0.8010
Epoch: 004, Train: 1.0000, Val: 0.7740, Test: 0.8010
Epoch: 005, Train: 1.0000, Val: 0.7740, Test: 0.8010
Epoch: 006, Train: 1.0000, Val: 0.7740, Test: 0.8010
Epoch: 007, Train: 1.0000, Val: 0.7820, Test: 0.8050
Epoch: 008, Train: 1.0000, Val: 0.7820, Test: 0.8050
Epoch: 009, Train: 0.9929, Val: 0.7820, Test: 0.8050
Epoch: 010, Train: 0.9929, Val: 0.7820, Test: 0.8050
Epoch: 011, Train: 0.9929, Val: 0.7820, Test: 0.8050
Epoch: 012, Train: 0.9929, Val: 0.7820, Test: 0.8050
Epoch: 013, Train: 1.0000, Val: 0.7880, Test: 0.8290
Epoch: 014, Train: 1.0000, Val: 0.7880, Test: 0.8290
Epoch: 015, Train: 1.0000, Val: 0.7880, Test: 0.8290
Epoch: 016, Train: 1.0000, Val: 0.7880, Test: 0.8290
Epoch: 017, Train: 1.0000, Val: 0.7880, Test: 0.8290
Epoch: 018, Train: 1.0000, Val: 0.7880, Test: 0.8290
Epoch: 019, Train: 0.9929, Val: 0.7880, Test: 